In [1]:
# imports
import os
import re
import gensim
import string
import joblib
import pickle 
import warnings
import pyLDAvis
import numpy as np
import pandas as pd
import seaborn as sns
from gensim import corpora
from pprint import pprint
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from gensim.models import CoherenceModel
from wordcloud import STOPWORDS,WordCloud
import pyLDAvis.gensim_models as gensimvis
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from joblib import dump, load # used for saving and loading sklearn objects
from scipy.sparse import save_npz, load_npz # used for saving and loading sparse matrices
from sklearn.decomposition import NMF, LatentDirichletAllocation
warnings.filterwarnings("ignore")

c:\Users\f0x-tr0t\AppData\Local\Programs\Python\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
c:\Users\f0x-tr0t\AppData\Local\Programs\Python\Python310\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
c:\Users\f0x-tr0t\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [2]:
# import custom libraries and scripts
# sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(".")
sys.path.append("..")

from defaults import *
from extract_dataframe import read_json
from extract_dataframe import TweetDfExtractor
from clean_tweets_dataframe import Clean_Tweets

In [3]:
# read csv
tweets_df = pd.read_csv("../data/clean_data.csv")
tweets_df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,status_count,retweet_count,screen_name,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,clean_hashtags,cleaned_mentions
0,2022-08-07 22:31:20,Twitter for Android,RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,en,4,8097,2,i_ameztoy,i_ameztoy,20497,2621,unknown,"[{'text': 'City', 'indices': [132, 137]}]","[{'screen_name': 'i_ameztoy', 'name': 'Iban Am...",unknown,City,i_ameztoy
1,2022-08-07 22:31:16,Twitter for Android,RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,en,691,5831,201,ZIisq,ZIisq,65,272,unknown,"[{'text': 'China', 'indices': [18, 24]}, {'tex...","[{'screen_name': 'IndoPac_Info', 'name': 'Indo...",unknown,China,IndoPac_Info
2,2022-08-07 22:31:07,Twitter for Android,"China even cut off communication, they don't a...",0.000000e+00,0.000000,en,0,1627,0,Fin21Free,Fin21Free,85,392,unknown,"[{'text': 'XiJinping', 'indices': [127, 137]}]","[{'screen_name': 'ZelenskyyUa', 'name': 'Волод...",Netherlands,Taiwan,ZelenskyyUa
3,2022-08-07 22:31:06,Twitter for Android,"Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,en,0,1627,0,Fin21Free,Fin21Free,85,392,unknown,"[{'text': 'XiJinping', 'indices': [9, 19]}]",[],Netherlands,XiJinping,ChinaUncensored
4,2022-08-07 22:31:04,Twitter for iPhone,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,en,1521,18958,381,VizziniDolores,VizziniDolores,910,2608,unknown,[],"[{'screen_name': 'ChinaUncensored', 'name': 'C...","Ayent, Schweiz",XiJinping,benedictrogers


In [4]:
cleaner = Clean_Tweets(tweets_df.copy())

Automation in Action...!!!


### Sentiment analysis

In [5]:
# select necessary columns
cleanTweet = tweets_df[["original_text", "polarity", 'subjectivity']]
cleanTweet

,original_text,polarity,subjectivity
0,RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625
1,RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000
2,"China even cut off communication, they don't a...",0.000000e+00,0.000000
3,"Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000
4,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250
...,...,...,...
7435,RT @metesohtaoglu: 📌📸 Map of #China's possible...,0.000000e+00,1.000000
7436,RT @NEVERBOW: China is doing #exactly what #Ru...,2.500000e-01,0.250000
7437,Minister Wu is crystal clear in his @BBCNews i...,1.583333e-01,0.419444
7438,Reports say that #China is planning to seize #...,0.000000e+00,0.000000


In [6]:
def text_category(p):
    """
    converts polarity into sentiment category
    """
    if p > 0:
        return "positive"
    elif p < 0:
        return "negative"
    else:
        return "neutral"

In [7]:
cleanTweet["score"] = cleanTweet["polarity"].apply(text_category)
cleanTweet

,original_text,polarity,subjectivity,score
0,RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,negative
1,RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,negative
2,"China even cut off communication, they don't a...",0.000000e+00,0.000000,neutral
3,"Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,positive
4,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,negative
...,...,...,...,...
7435,RT @metesohtaoglu: 📌📸 Map of #China's possible...,0.000000e+00,1.000000,neutral
7436,RT @NEVERBOW: China is doing #exactly what #Ru...,2.500000e-01,0.250000,positive
7437,Minister Wu is crystal clear in his @BBCNews i...,1.583333e-01,0.419444,positive
7438,Reports say that #China is planning to seize #...,0.000000e+00,0.000000,neutral


In [8]:
sentiment = cleanTweet['score'].value_counts()
sentiment

neutral     2894
positive    2768
negative    1778
Name: score, dtype: int64

#### as we can see there are:
    * 2894 neutral sentiment
    * 2768 positive sentiment
    * 1778 negative sentiment
#### tweets

#### removing neutral sentiments

In [9]:
# remove neutral tweets
cleanTweet = cleanTweet[cleanTweet['score'] != 'neutral']
cleanTweet

,original_text,polarity,subjectivity,score
0,RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,negative
1,RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,negative
3,"Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,positive
4,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,negative
5,RT @benedictrogers: We must not let this happe...,2.000000e-01,0.500000,positive
...,...,...,...,...
7432,@kiwispillow @dbongino No ones buying this non...,1.000000e-01,1.000000,positive
7433,Senior Taiwanese missile development official ...,-2.000000e-01,0.400000,negative
7436,RT @NEVERBOW: China is doing #exactly what #Ru...,2.500000e-01,0.250000,positive
7437,Minister Wu is crystal clear in his @BBCNews i...,1.583333e-01,0.419444,positive


In [10]:
# add score map column
def scoremap(score):
    """
    maps sentiment score to 0 and 1
    """
    if score == "positive":
        return 1
    else:
        return 0

In [11]:
cleanTweet['scoremap'] = cleanTweet["score"].map(scoremap)
cleanTweet

,original_text,polarity,subjectivity,score,scoremap
0,RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,negative,0
1,RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,negative,0
3,"Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,positive,1
4,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,negative,0
5,RT @benedictrogers: We must not let this happe...,2.000000e-01,0.500000,positive,1
...,...,...,...,...,...
7432,@kiwispillow @dbongino No ones buying this non...,1.000000e-01,1.000000,positive,1
7433,Senior Taiwanese missile development official ...,-2.000000e-01,0.400000,negative,0
7436,RT @NEVERBOW: China is doing #exactly what #Ru...,2.500000e-01,0.250000,positive,1
7437,Minister Wu is crystal clear in his @BBCNews i...,1.583333e-01,0.419444,positive,1


In [12]:
(X, y) = cleanTweet['original_text'], cleanTweet['scoremap']
trigram_vectorizer = CountVectorizer(ngram_range=(1, 3))
trigram_vectorizer.fit(X.values)

CountVectorizer(ngram_range=(1, 3))

In [13]:
X_trigram_vectorizer = trigram_vectorizer.transform(X.values)

In [14]:
def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.75, stratify=y
    )

    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)
    print(f'{title}\nTrain score: {round(train_score, 2)} ; Test score: {round(test_score, 2)}\n')

In [15]:
train_and_show_scores(X_trigram_vectorizer, y.values, 'Trigram Counts')

Trigram Counts
Train score: 1.0 ; Test score: 0.81



### Topic modeling

In [16]:
tweets_df.columns

Index(['created_at', 'source', 'original_text', 'polarity', 'subjectivity',
       'lang', 'favorite_count', 'status_count', 'retweet_count',
       'screen_name', 'original_author', 'followers_count', 'friends_count',
       'possibly_sensitive', 'hashtags', 'user_mentions', 'place',
       'clean_hashtags', 'cleaned_mentions'],
      dtype='object')

In [17]:
#text Preprocessing
tweets_df['original_text']=tweets_df['original_text'].astype(str)
tweets_df['original_text'] = tweets_df['original_text'].apply(lambda x: x.lower())
tweets_df['original_text']= tweets_df['original_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))

In [18]:
#Converting tweets to list of words For feature engineering
sentence_list = [tweet for tweet in tweets_df['original_text']]
word_list = [sent.split() for sent in sentence_list]

In [19]:
# word_list

In [20]:
len(word_list), type(word_list)

(7440, list)

In [21]:
#Create dictionary which contains Id and word 
id2word = corpora.Dictionary(word_list)
corpus= [id2word.doc2bow(tweet) for tweet in word_list]

In [22]:
print(id2word)

Dictionary<25113 unique tokens: ['city', 'coast', 'district', 'extra', 'focus']...>


In [23]:
# corpus

In [24]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('taiwan', 0.061315455),
   ('the', 0.05964462),
   ('to', 0.040232584),
   ('china', 0.035543233),
   ('of', 0.030504225),
   ('and', 0.027452258),
   ('in', 0.023780696),
   ('is', 0.022769393),
   ('rt', 0.0222886),
   ('a', 0.022058424)]),
 (1,
  [('you', 0.025121083),
   ('we', 0.019022327),
   ('i', 0.017762445),
   ('taiwanstraitscrisis', 0.010783229),
   ('their', 0.010219298),
   ('your', 0.010165102),
   ('or', 0.009268895),
   ('if', 0.008947265),
   ('now', 0.008390369),
   ('time', 0.008370824)]),
 (2,
  [('russia', 0.024044784),
   ('war', 0.02392964),
   ('ukraine', 0.023536507),
   ('usa', 0.022903256),
   ('speakerpelosi', 0.010289242),
   ('israel', 0.009817532),
   ('against', 0.008910389),
   ('japan', 0.008573305),
   ('taiwanchina', 0.007948937),
   ('america', 0.0077911797)]),
 (3,
  [('taiwanese', 0.02229605),
   ('who', 0.013180205),
   ('missile', 0.011471781),
   ('found', 0.009427182),
   ('global', 0.008946125),
   ('missiles', 0.0082056075),
   ('a

In [26]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('Perplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

Perplexity:  -8.240797422616497


In [27]:
print('Lda model Coherence Score/Accuracy on Tweets: ', coherence_lda)

Lda model Coherence Score/Accuracy on Tweets:  0.37401248504713375


In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.382428  0.037995       1        1  59.491238
1     -0.046932 -0.286522       2        1  14.740237
2     -0.113351  0.119001       3        1  10.764600
4     -0.114193  0.081077       4        1   8.891490
3     -0.107952  0.048449       5        1   6.112434, topic_info=           Term         Freq        Total Category  logprob  loglift
37       taiwan  6744.000000  6744.000000  Default  30.0000  30.0000
19          the  6561.000000  6561.000000  Default  29.0000  29.0000
57           to  4425.000000  4425.000000  Default  28.0000  28.0000
42        china  3910.000000  3910.000000  Default  27.0000  27.0000
12           of  3355.000000  3355.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
5226   shipping    37.550894    38.297121   Topic5  -5.7070   2.7752
7419  ballistic    34.119871    34.840787   Topic5  -5.8028   2.7739
4417      range    34.331125    35.060083   Topic5  -5.7966   2.7738
4480       head    51.462266    76.383435   Topic5  -5.3918   2.3999
431     taiwans    55.225962   166.534773   Topic5  -5.3212   1.6911

[203 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1497      5  0.970590        11
61        1  0.938233         a
61        2  0.061492         a
463       3  0.994029   against
427       5  0.987648  aircraft
...     ...       ...       ...
82        2  0.975940       you
82        4  0.024220       you
584       2  0.997379      your
1062      3  0.993785   youtube
2170      3  0.989779        🇨🇳

[194 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 5, 4])

save the model

In [30]:
joblib.dump(lda_model, '../models/lda_model.pkl')

['../models/lda_model.pkl']